In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 901kB 19.9MB/s 
     |████████████████████████████████| 3.3MB 31.2MB/s 


In [ ]:
import datetime
import json
import urllib.parse
from bs4 import BeautifulSoup
from transformers import pipeline
import requests
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [13]:
URL = "https://www.imsnsit.org/imsnsit/notifications.php"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

base_id = "id" #base id
table_name = "Table%201" #table name
url = "https://api.airtable.com/v0/" + base_id + "/" + table_name + "?fields%5B%5D=Title&maxRecords=1&sort%5B0%5D%5Bfield%5D=Created&sort%5B0%5D%5Bdirection%5D=desc"
headers = {"Authorization" : "Bearer" + " key*****", #enter api key
 "Content-Type" : "application/json"}
#params = {"maxRecords" : 1, "sort" : [{"field": "Date", "direction": "desc"}]}
response = requests.get(url, headers=headers)
airtable_response = response.json()
print(airtable_response)
airtable_records = airtable_response['records']
try:
  last_record = airtable_records[0]['fields']['Title']
except:
  last_record = None


sequences = []
table = soup.findAll('a', attrs = {'title':'NOTICES / CIRCULARS'}) #get table from ims
candidate_labels1 = ['1 semester', '2 semester', '3 semester', '4 semester', '5 semester', '6 semester', '7 semester', '8 semester']
candidate_labels2 = ['b.tech', 'm.tech', 'bba', 'mba', 'bftech']
candidate_labels3 = ['Electronics/Communications/Instrumentation', 'BioTech', 'Computer Science', 'Mechanical Engineering', 'English', 'Mathematics', 'Civil']
candidate_labels4 = ['registration', 'rules', 'forms', 'classroom codes', 'examinations', 'workshops', 'time table']
print(table)
if last_record:
  print(last_record)
  for i in range(len(table)):
    if table[i].font.text == last_record:
      table = table[:i]
      break
table = table[::-1]
for row in range(len(table)):
  print("Row {} of {}".format(row+1, len(table)))
  row = table[row]
  sequence = row.font.text
  if sequence == last_record:
    print("Airtable already updated!")
    break
  date = row.parent
  date = date.parent
  date = date.findAll('td')
  for i in date:
    date = i.font.text.strip()
    break
  date_obj = datetime.datetime.strptime(date, '%d-%m-%Y')

  y1 = classifier(sequence, candidate_labels1, multi_label=True) #Semester
  y1 = y1['labels']
  for i in range(len(y1)):
    y1[i] = "Semester "+y1[i][0]
  y2 = classifier(sequence, candidate_labels2, multi_label=True) #Course Type
  y2 = y2['labels']
  for i in range(len(y2)):
    if y2[i] == 'b.tech':
      y2[i] = 'B. Tech'
    elif y2[i] == 'm.tech':
      y2[i] = 'M. Tech'
    elif y2[i] == 'bftech':
      y2[i] = 'B. F. Tech'
    elif y2[i] == 'bba':
      y2[i] = 'BBA'
    elif y2[i] == 'mba':
      y2[i] = 'MBA'
  y3 = classifier(sequence, candidate_labels3, multi_label=True) #Subject
  y3 = y3['labels']
  y4 = classifier(sequence, candidate_labels4, multi_label=True) #General
  y4 = y4['labels']
  upload_data = {'Title': sequence,
                 'URL': "https://imsnsit.glitch.me/?url="+urllib.parse.quote(row['href']),
                 'Semester': y1[:2],
                 'Course Type': y2[:3],
                 'Subject': y3[:2],
                 'General': y4[:3],
                 'Date': str(date_obj.date())
                 }
  upload_dict = {"records" : [{"fields" : upload_data}], 
               "typecast" : False}
  upload_json = json.dumps(upload_dict)
  response = requests.post(url, data=upload_json, headers=headers)
  sequences.append(sequence)
  print(response.json())

{'records': [{'id': 'recv3kyVHC9a0oinD', 'fields': {'Title': 'GOOGLE CLASSROOM CODES OF WEST CAMPUS COURSES _SECOND SEMESTER'}, 'createdTime': '2021-04-19T17:37:40.000Z'}]}
[<a href="https://www.imsnsit.org/imsnsit/plum_url.php?QeHb+FgMobEDbcTPOvDfhvfkb2Smb5zdhZc4/vC5po8ISEKcJWS4jS3w2UqVvr5F5fIs09NBXvMMtVi3O7eZPsCtw/a9ORkdAYOjWSwaUIGMjjlKsh2HiJ2wIpgMmpMjmcOhTuyIjPgY66PjnN7Lbm4BVphWgPM+IR002zz1aspFWq67L5e58QN84YRSAe2suNAWs4C8pRRZniggnwBSXQ==" style="text-decoration:none;" target="_BLANK" title="NOTICES / CIRCULARS"><font size="3">Classroom codes of the 2nd Semester Subjects offered by ECE Department</font></a>, <a href="https://www.imsnsit.org/imsnsit/plum_url.php?j/WgjSPXGiUbbYNrNZ4kzDD2qi/ypB/+m9U0FIOtY2bISsp/zl8iKLypaGw/BhMwhxn9NpSoLkdj2HSOFLCclqGTxLCRTlNpwfX5DQZQHwSMPyxr3YQTnHVfhz8gayYV0lBII7eFtne+Jn6D5AXruhxgCZ3Ggh1Cax/awnExZWLL3igJYb/at+BSQKaWHDk4+zlptMVbDXlZU7TXlWT92fqPsf3VKO0PxxRK0vYKk7g=" style="text-decoration:none;" target="_BLANK" title="NOTICES / CIRCULARS"><font size="3">C